In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from pysal.lib import weights
from splot.libpysal import plot_spatial_weights
from esda.moran import Moran, Moran_Local
from splot.esda import moran_scatterplot, plot_local_autocorrelation, lisa_cluster
import matplotlib.pyplot as plt
import folium
import matplotlib.colors as colors

import warnings
warnings.filterwarnings("ignore")

In [ ]:
censo_es = gpd.GeoDataFrame.from_file('basemunicipal.geojson')
censo_es['População'] = censo_es['População'].str.replace(r'\D', '')
censo_es['População'] = censo_es['População'].astype(int)
censo_es.head()

In [ ]:
#Obtendo o valor do PIB real, uma vez que o valor no dataset original se encontra em milhoes
censo_es['PIB2020'] = censo_es['PIB2020'] *1000000
#Convertendo area de KM2 para M2
censo_es['aream2'] = censo_es['areaKm2'] *1000000
#Calculando o PIB percapita dos municipios
censo_es['PIB_percap'] = censo_es['PIB2020'] / censo_es['População']
#Calculando a populacao por KM2
censo_es['pop_km2'] = censo_es['População']  / censo_es['areaKm2']
censo_es.head()

In [ ]:
# Plotando choropleth map da coluna Populacao/m2
censo_es.explore(
    tiles='cartodbpositron',
    column='pop_km2',
    height='60%',
    width='60%',
    scheme='naturalbreaks',
    cmap='Oranges',
    style_kwds={
        'stroke': True,
        'edgecolor': 'k',
        'linewidth': 0.03,
        'fillOpacity': 1
        },
    legend_kwds={
        'colorbar': False,
        'fmt': '{:.0f}'
        }
    )

In [ ]:
# Plotting choropleth map da coluna Pibpercap
censo_es.explore(
    tiles='cartodbpositron',
    column='PIB_percap',
    height='60%',
    width='60%',
    scheme='naturalbreaks',
    cmap='Oranges',
    style_kwds={
        'stroke': True,
        'edgecolor': 'k',
        'linewidth': 0.03,
        'fillOpacity': 1
        },
    legend_kwds={
        'colorbar': False,
        'fmt': '{:.0f}'
        }
    )

In [ ]:
# Construindo spatial weights object
w = weights.contiguity.Queen.from_dataframe(censo_es)

# Plotando spatial weights
plot_spatial_weights(w, censo_es);

In [ ]:
# Transformando weights em binario (Se 1 = is neighbor, 0 = not neighbor)
w.transform = "B"

# Showing neighbors indexes
w.neighbors

In [ ]:
# Showing weights
w.weights

In [ ]:
# Calculando Spatial Lag
censo_es['w_pop'] = weights.lag_spatial(w, censo_es['População'])
censo_es['w_PIB'] = weights.lag_spatial(w, censo_es['PIB2020'])
censo_es['w_PIBperc'] = weights.lag_spatial(w, censo_es['PIB_percap'])
censo_es['w_popkm2'] = weights.lag_spatial(w, censo_es['pop_km2'])

censo_es[['geocodigo', 'População', 'w_pop', 'PIB2020','PIB_percap','w_PIB', 'w_popkm2', 'w_PIBperc']].head()

In [ ]:
# Variable of interest (populacao por km2)
y_popper_count = censo_es['pop_km2']

# Calculando Moran I statistic
moran = Moran(y_popper_count, w)

# Moran's I statistic
moran.I


In [ ]:
moran.p_sim

In [ ]:
# Local Moran's I
popper_count_local_moran = Moran_Local(y_popper_count, w)

# Plotting Local Moran's I scatterplot of pop_count
fig, ax = moran_scatterplot(popper_count_local_moran, p=0.05);

plt.text(1.95, 1, 'HH', fontsize=25)
plt.text(1.95, -1.0, 'HL', fontsize=25)
plt.text(-1.5, 1, 'LH', fontsize=25)
plt.text(-1.5, -1, 'LL', fontsize=25)
plt.show()

In [ ]:
# Variable of interest (PIB2020)
y_total_pipercap = censo_es['PIB_percap']
# Calculating Moran's I statistic
moran = Moran(y_total_pipercap, w)

# Moran's I statistic
moran.I

In [ ]:
moran.p_sim

In [ ]:

moran_scatterplot(moran);

In [ ]:
# Local Moran's I
total_pipercap = Moran_Local(y_total_pipercap, w)

# Plotting Local Moran's I scatterplot of pop_count
fig, ax = moran_scatterplot(popper_count_local_moran, p=0.05);

plt.text(1.95, 1, 'HH', fontsize=25)
plt.text(1.95, -1.0, 'HL', fontsize=25)
plt.text(-1.5, 1, 'LH', fontsize=25)
plt.text(-1.5, -1, 'LL', fontsize=25)
plt.show()

In [ ]:
# creating column with local_moran classification
censo_es['popper_count_local_moran'] = popper_count_local_moran.q

# Dict to map local moran's classification codes
local_moran_classification = {1: 'HH', 2: 'LH', 3: 'LL', 4: 'HL'}

# Mapping local moran's classification codes

censo_es['popper_count_local_moran'] = censo_es['popper_count_local_moran'].map(local_moran_classification)

# p-value for each observation/neighbor pair

censo_es['popper_count_local_moran_p_sim'] = popper_count_local_moran.p_sim

# If p-value > 0.05 it is not statistical significant

censo_es['popper_count_local_moran'] = np.where(censo_es['popper_count_local_moran_p_sim'] > 0.05, 'ns', censo_es['popper_count_local_moran'])
censo_es.head()

In [ ]:
# Plotting Local Moran's I classificacao populacao por km2
censo_es.explore(
    tiles='cartodbpositron',
    column='popper_count_local_moran',
    height='70%',
    width='70%',
    cmap=[
    '#D7191C', # Red
    '#FDAE61', # Orange
    '#ABD9E9', # Light Blue
    '#2C7BB6', # Blue
    '#D3D3D3'  # Grey
    ],
    style_kwds={
        'stroke': True,
        'edgecolor': 'k',
        'linewidth': 0.03,
        'fillOpacity': 1
        
        },
    )

In [ ]:
# creating column with local_moran classification
censo_es['total_pipercap'] = total_pipercap.q

# Dict to map local moran's classification codes
local_moran_classification = {1: 'HH', 2: 'LH', 3: 'LL', 4: 'HL'}

# Mapping local moran's classification codes
censo_es['total_pipercap'] = censo_es['total_pipercap'].map(local_moran_classification)

# p-value for each observation/neighbor pair
censo_es['total_pipercapp_sim'] = total_pipercap.p_sim

# If p-value > 0.05 it is not statistical significant
censo_es['total_pipercap'] = np.where(censo_es['total_pipercapp_sim'] > 0.05, 'ns', censo_es['total_pipercap'])

# Plotting Local Moran's I classification map of pop_count column
censo_es.explore(
    tiles='cartodbpositron',
    column='total_pipercap',
    height='70%',
    width='70%',
    cmap=[
    '#D7191C', # Red
    '#FDAE61', # Orange
    '#ABD9E9', # Light Blue
    '#2C7BB6', # Blue
    '#D3D3D3'  # Grey
    ],
    style_kwds={
        'stroke': True,
        'edgecolor': 'k',
        'linewidth': 0.03,
        'fillOpacity': 1
        
        },
    )